In [17]:
!pip3 install requests pybreaker tenacity

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [1]:
import requests
import pybreaker
from tenacity import retry, stop_after_attempt, wait_exponential_jitter, retry_if_exception_type
from requests.exceptions import RequestException, Timeout


In [2]:
BACKEND_URL = "http://127.0.0.1:60887/api/first"

In [3]:
# Circuit Breaker Listener to print state changes
class PrintStateChangeListener(pybreaker.CircuitBreakerListener):
    def state_change(self, cb, old_state, new_state):
        print(f"Circuit breaker state changed from {old_state.name} to {new_state.name}")

# Create circuit breaker with listener
circuit_breaker = pybreaker.CircuitBreaker(
    fail_max=1,           # Open circuit after 2 failures
    reset_timeout=.10,     # Stay open for 10 seconds before trying half-open
    listeners=[PrintStateChangeListener()]
)

In [4]:
# Define fallback response when circuit breaker is open
def fallback_response():
    print("Circuit breaker is open. Returning fallback response.")
    return {"message": "Service temporarily unavailable, please try later."}

# Retry decorator: retry on RequestException or Timeout with exponential backoff + jitter
@retry(
    stop=stop_after_attempt(4),                      # Max 4 attempts
    wait=wait_exponential_jitter(initial=1, max=10),# Backoff with jitter between 1 and 10 seconds
    retry=retry_if_exception_type((RequestException, Timeout)),
    reraise=True
)
def make_request():
    print("Attempting to call backend...")
    response = requests.get(BACKEND_URL, timeout=3)  # 3 seconds timeout
    response.raise_for_status()                       # Raise HTTPError for bad status codes
    return response.json()

# Call backend with circuit breaker protection and retry logic
def call_backend():
    for i in range(30):
        print(f"\nRequest attempt {i+1}:")
        try:
            # Use circuit breaker to protect the call
            result = circuit_breaker.call(make_request)
            print("Backend response:", result)
        except pybreaker.CircuitBreakerError:
            # Circuit breaker is open, fallback response
            print("Circuit breaker is open. Skipping backend call.")           
            fallback_response()
        except RequestException as e:
            # Request failed even after retries
            print(f"Request failed after retries: {e}")

In [6]:
call_backend()


Request attempt 1:
Attempting to call backend...
Backend response: {'message': 'Hello World 1'}

Request attempt 2:
Attempting to call backend...
Backend response: {'message': 'Hello World 1'}

Request attempt 3:
Attempting to call backend...
Attempting to call backend...
Attempting to call backend...
Backend response: {'message': 'Hello World 1'}

Request attempt 4:
Attempting to call backend...
Backend response: {'message': 'Hello World 1'}

Request attempt 5:
Attempting to call backend...
Attempting to call backend...
Backend response: {'message': 'Hello World 1'}

Request attempt 6:
Attempting to call backend...
Backend response: {'message': 'Hello World 1'}

Request attempt 7:
Attempting to call backend...
Backend response: {'message': 'Hello World 1'}

Request attempt 8:
Attempting to call backend...
Backend response: {'message': 'Hello World 1'}

Request attempt 9:
Attempting to call backend...
Backend response: {'message': 'Hello World 1'}

Request attempt 10:
Attempting to ca